In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

### Turn off package warning
### warnings.filterwarnings(action='once')

import warnings
warnings.filterwarnings('ignore')

In [23]:
training_data = pd.read_csv('train.csv', encoding = "ISO-8859-1")
    
training_data = training_data.dropna(axis='index', subset=['compliance'])     
    
X_train = training_data.drop('compliance', axis=1)
    
listoffloatvars = ['fine_amount',
'admin_fee',
'state_fee', 
'late_fee', 
'discount_amount', 
'clean_up_cost', 
'judgment_amount', 
'payment_amount',
'balance_due',]
#'violation_street_number'

listoftestfloats = ['fine_amount',
'admin_fee',
'state_fee', 
'late_fee', 
'discount_amount', 
'clean_up_cost', 
'judgment_amount']

X_final_model = training_data[listoftestfloats]
    
X_train_ori = training_data[listoffloatvars]
    
y_train_ori  = training_data['compliance']
    
X_train, X_test, y_train, y_test =  train_test_split(X_train_ori, y_train_ori)

In [24]:
from sklearn.metrics import roc_auc_score, recall_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.neighbors import NearestNeighbors
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

def model_selection():

    models = []
    models.append(('SGD', SGDClassifier()))
    models.append(('LR', LogisticRegression()))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('CART', DecisionTreeClassifier(max_depth = 10)))
    models.append(('NB', GaussianNB()))
    
    results = []
    names = []
    scoring = 'roc_auc'
    for name, model in models:
        kfold = KFold(n_splits=10, random_state=50)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

In [25]:
model_selection()

SGD: 0.893636 (0.048659)
LR: 0.919845 (0.003976)
LDA: 0.916098 (0.002475)
CART: 0.977031 (0.001282)
NB: 0.907025 (0.007124)


In [11]:
def empty_model():
    clf = DecisionTreeClassifier()
    scoringg = (cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=10)).mean()
    return clf

In [17]:
def optimizing():
    #criterion : 'gini'
    #splitter : 'best'
    params = {'max_depth' : [6,7,8,9,10] , 'min_samples_split' : [6,7,8,9,10], 'min_samples_leaf' : [1,2,3]}
    gs = GridSearchCV(empty_model(), param_grid = params, scoring='roc_auc', cv = 10 )
    gs = gs.fit(X_train, y_train)
    return (gs.best_score_, gs.best_params_ ) 

In [18]:
optimizing()

(0.97767658171557437,
 {'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 8})

In [19]:
def final_model_probabilities():
    test_data = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    clf = DecisionTreeClassifier(max_depth=9, min_samples_leaf=1, min_samples_split=8).fit(X_final_model, y_train_ori)
    return clf.predict_proba(test_data[listoftestfloats])[:,1]

In [20]:
def tick_ids():
    test_data = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    return pd.DataFrame(test_data['ticket_id'])

In [21]:
def blight_model():
    ids = tick_ids()
    ids['Probability'] = final_model_probabilities()
    # ids.set_index('ticket_id')[ids.set_index('ticket_id')['Probability'] > 0.51]
    return ids.set_index('ticket_id')

blight_model()

,Probability
ticket_id,
284932,0.060683
285362,0.023744
285361,0.069591
285338,0.060683
285346,0.069591
285345,0.060683
285347,0.057378
285342,0.376740
285530,0.023744
